In [ ]:
# Copyright 2018 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

## Training Demonstration for Computer Vision / Color - Fruits360

This demonstration will use the Frutis360 dataset. This is a dataset provided by Horea Mures¸Faculty of Mathematics and Computer Science, Bolyai University, Romania and MiHai Oltean, Faculty of Exact Sciences and
Engineering, University of Alba Iulia, Romania. The dataset consists of 81 classes of fruits and their varieties. The dataset was originally used for their research on their published paper *Fruit Recognition from images using Deep Learning*, 2018, https://arxiv.org/pdf/1712.00580.pdf . The dataset consists of 55,000. The images are split into a folder for training and a second folder for test (evaluation).

Below is the description on how they prepared the images:

    Fruits were planted in the shaft of a low speed motor (3 rpm) and a short movie of 20 seconds was recorded.

    A Logitech C920 camera was used for filming the fruits. This is one of the best webcams available.

    Behind the fruits we placed a white sheet of paper as background.

    However due to the variations in the lighting conditions, the background was not uniform and we wrote a dedicated algorithm which extract the fruit from the background. This algorithm is of flood fill type: we start from each edge of the image and we mark all pixels there, then we mark all pixels found in the neighborhood of the already marked pixels for which the distance between colors is less than a prescribed value. We repeat the previous step until no more pixels can be marked.

    All marked pixels are considered as being background (which is then filled with white) and the rest of pixels are considered as belonging to the object.

    The maximum value for the distance between 2 neighbor pixels is a parameter of the algorithm and is set (by trial and error) for each movie.



## Prerequistes

The following needs to be pre-installed:

        openCV : pip install opencv-python
        numpy  : pip install numpy
        ipynb  : pip install import-ipynb

In [ ]:
import cv2
import numpy as np
import import_ipynb

### Download the Dataset

The Fruits360 dataset will need to be downloaded to the same directory (folder) as this notebook.

A zip file (compressed) of the dataset can be obtained at this location:

https://pantheon.corp.google.com/storage/browser/cloud-samples-data/air/fruits360/

## ML Pipeline Chain

The following ML Pipelines will be chained together for this demonstration

        frutis360- > openCV -> hdf5 -> model_keras

### Process Image Files into Machine Learning Data using OpenCV module

In [ ]:
# Import the openCV ML pipeline
import openCV

#### Process Training Set

In [ ]:
# Process the on-disk training set of images to in-memory set of machine learning ready data
dataset_train = openCV.load_directory('fruits360/Training', colorspace=openCV.COLOR, resize=(100,100), flatten=False, concurrent=4, verbose=True)

Dataset should be 81 collections (fruits and there varieties).
Each collection should consist of a set of three entries: data, labels, and errors.

In [ ]:
print( "Number of collections:", len(dataset_train) )
print( "Number of sets in a collection:", len(dataset_train[0]))

The first collection should have the label (fruit) 'Avocado' and consist of 427 images.

In [ ]:
print("Number of images:", len(dataset_train[0][0]))
print("Label for collection:", dataset_train[0][1])

In [ ]:
print("Shape of Preprocessed Image", dataset_train[0][0][0].shape)

#### Processing Test Set

In [ ]:
# Process the on-disk test set of images to in-memory set of machine learning ready data
dataset_test = openCV.load_directory('fruits360/Test', colorspace=openCV.COLOR, resize=(100,100), flatten=False, concurrent=4, verbose=True)

In [ ]:
print( "Number of collections:", len(dataset_test) )
print( "Number of sets in a collection:", len(dataset_test[0]))

The first collection should have the label (fruit) 'Avocado' and consist of 143 images.

In [ ]:
print("Number of images:", len(dataset_test[0][0]))
print("Label for collection:", dataset_test[0][1])

### Store Machine Learning Ready (preprocessed images) data into HDF5 storage

In [ ]:
# Import the HDF5 storage ML pipeline
import hdf5

In [ ]:
# Store the training machine learning ready data to HDF5
hdf5.store_dataset('fruits360-training', dataset_train, verbose=True)

In [ ]:
import os
print("HDF5 file size:", int( os.path.getsize('fruits360-training.h5') / (1024 * 1024) ), "MB")

In [ ]:
# Store the test machine learning ready data to HDF5
hdf5.store_dataset('fruits360-test', dataset_test, verbose=True)

In [ ]:
import os
print("HDF5 file size:", int( os.path.getsize('fruits360-test.h5') / (1024 * 1024) ), "MB")

### Construct CNN using Keras

The CNN below is constructed according to the first method used by Muresan and Oltean in their published paper, with the addition of a dropout layer, which was not used in their paper. Additionally, in their paper they tried other CNN configurations, conversation to grayscale and HSV colorspace and image augmentation.

In [ ]:
# Import the Keras CNN Model ML pipeline
import model_keras

In [ ]:
# Construct a CNN with input layer of NN of:
# Convolutional Layer of 16 filters with input vector (100, 100, 3)
# Convolutional Layer of 32 filters
# Convolutional Layer of 64 filters
# Convolutional Layer of 128 filters
# Neural Network Layer of 1024 nodes and 0.25% dropout
# Nerual Network Layer of 256 nodes
# Output Layer with 81 nodes (classes)
model = model_keras.construct_cnn( (100, 100, 3), 81, n_filters=(16, 32, 64, 128), n_nodes=(1024, 256), dropout=(0.25,0))

### Train the Model

In [ ]:
# load the dataset back into memory
collections_train, labels_train, classes = hdf5.load_dataset('fruits360-training')
collections_test, labels_test, classes = hdf5.load_dataset('fruits360-test')

In [ ]:
print("Training")
print("Images", type(collections_train), len(collections_train))
print("Labels", type(labels_train), len(labels_train))
print("Classes", classes)

print("Test")
print("Images", type(collections_test), len(collections_test))
print("Labels", type(labels_test), len(labels_test))

During training (in verbose mode), each epoch will output the current accuracy on the training data (acc) and accuracy on the testing data (val_acc).

*Best Practices*
1. Once the value of val_acc levels off (stops improving) you should stop training; otherwise the model may overfit.

2. If there is a high value for acc and low value for val_acc, the model is likely overfitted. Things to try:
        A. Add higher dropout or dropout to more layers.
        B. Reduce the number of nodes.
        
3. If you increase the batch size, the training time per epoch is reduced. Common practice is to set (mini) batch sizes between 32 and 256.

In [ ]:
# Train the model
accuracy = model_keras.train_cnn(model, collections_train, labels_train, collections_test, labels_test, epochs=10, batch_size=256, verbose=True)

In [ ]:
# Display the accuracy
print(accuracy)

### Save the Model

In [ ]:
# Save the model
model.save('fruits360.model.h5')